# Runnable

In [3]:
from abc import ABC, abstractmethod

class Runnable(ABC):

    @abstractmethod
    def invoke(input_data):
        pass

In [34]:
import random
class DummyLLM(Runnable):
     def __init__(self):
         print("LLM Object created.")

     def invoke(self, prompt):
         response_list = [
             'Anakin didn\'t had high ground.',
             'Obi-Wan cheated',
             'Tragedy of Darth Plagueis The wise'
         ]
         return  {'response': random.choice(response_list)}

     def predict(self, prompt):
         response_list = [
             'Anakin didn\'t had high ground.',
             'Obi-Wan cheated',
             'Tragedy of Darth Plagueis The wise'
         ]

         return  {'response': random.choice(response_list), 'warning': 'This method is going to be deprecated.'}

class DummyPromptTemplate(Runnable):
    def __init__(self, input_variable, template):
        self.input_variable = input_variable
        self.template = template

    def invoke(self, input_dict):
        return self.template.format(**input_dict)

    def format(self, input_dict):
        return f"{self.template.format(**input_dict)} , This method is going to be deprecated"

class DummyStrOutputParser(Runnable):
    def __init__(self):
        pass      

    def invoke(self, input_dict):
        return input_dict['response']
        

In [14]:
llm = DummyLLM()
print(llm.invoke("What is capital of Corosaunt?"))
print(llm.predict("What is capital of Tatoinne"))

LLM Object created.
{'response': 'Tragedy of Darth Plagueis The wise'}
{'response': "Anakin didn't had high ground.", 'warning': 'This method is going to be deprecated.'}


In [20]:
template = DummyPromptTemplate(
    template="What is the capital of {topic}",
    input_variable=['topic']
)

prompt = template.invoke({"topic":"Corosaunt"})

print(llm.invoke(prompt))

{'response': 'Obi-Wan cheated'}


- Both DummyPromptTemplate and DummyLLM has been standardized

## Class to connect multiple chains

In [23]:
class RunnableConnector(Runnable):
    def __init__(self, runnable_list):
        self.runnable_list = runnable_list

    def invoke(self, input_data):
        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)

        return input_data

In [35]:
parser = DummyStrOutputParser()

In [36]:
chain = RunnableConnector([template, llm, parser])

In [37]:
chain.invoke({"topic":"Corosaunt"})

'Tragedy of Darth Plagueis The wise'